In [60]:
#### specify path of Julia Code for 2D Models ####
### By default the working directory ./2D_Models ###
path = string( pwd(), "/2D_Models/")

"/home/jakob/Dokumente/Repositories/Modeling_Framework/2D_Models/"

In [61]:
##### include 2D Modelling Framework ####
include(string(path, "./2D_model_library.jl"));


########## OPERATIONAL ##########
##### how to chose the actual velocity? #####

#op_model = "Constant"
#op_model = "Collision_Free_Speed"
#op_model = "RVO"
#op_model = "SocialForce"
#op_model = "AV_Model"
#op_model = "AV_IN_Model"
#op_model = "PowerLaw"
#op_model = "IN_Costfunction"
#op_model = "IN_Model"
#op_model = "SocialForce"
#op_model = "SocialForce_Elliptical_A"
#op_model = "SocialForce_Elliptical_B"
op_model = "SocialForce_CollisionPrediction"


include_op_model(op_model, path)

########## TACTICAL ##########
#### how to chose the preferred velocity? ###

tact_model = "Constant"
#tact_model = "IN_Model"

include_tact_model(tact_model, path)


########## STRATEGIC ##########
### how to chose the desired velocity ###

strat_model = "Constant"
#strat_model = "Towards_Goal"
#strat_model = "Two_Goals"


include_strat_model(strat_model, path)


########## OPTIMIZATION SCHEME ##########
### If optimization of a cost-function is included, how to solve? ###

#optimization_scheme = "Regular_Sampling"
optimization_scheme = "Random_Sampling"

include_optimization_scheme(optimization_scheme, path)


##### which update scheme? #####

update = "Parallel_Update"
#update = "Step_Update"

include_update(update, path)


##### which order? i.e. Acceleration or Velocity based? ####
#order = "first"
order = "second"

include_order(order, path)


##### PERCEPTIONAL MODEL ####
perception_model = "None"
#perception_model = "Only_in_Front"

include_perception_model(perception_model, path)

The following parameters were given in zanlungo_SocialForceModel_2011:
[2.0, 1.7, 0.0, 0.0, 0.0, 0.0, 0.0, 0.66, 1.71, 0.0, 0.0, 0.0, 0.71, 0.0]
v_max, v_des, T, T2, l, step_time, τ_A, τ_R, α, β, ζ_h, ζ_v, r, ϕ


In [62]:
#### parameters of the agents ####
p = [2.0, 1.7, 0.0, 0.0, 0.3, 0.0, 0.0, 0.66, 1.71, 0.0, 0.0, 0.0, 0.71, 0.0]
# ModelParameter: v_max, v_des, T, T2, l, step_time, τ_A, τ_R, α, β, ζ_h, ζ_v, r, ϕ
p_desc = "v_max, v_des, T, T2, l, step_time, τ_A, τ_R, α, β, ζ_h, ζ_v, r, ϕ"

### of the system ###
N = 2
system_size = (5.0, 5.0) #important for periodic boundaries
#system_size = (35.0, 35.0) #important for periodic boundaries

### of the simulation ###
sim_p = [0.0, 5.0, 0.05, 0.01, 10.0] 
sim_p_desc = "t_relax, t_max, dt_save, dt, r"

### initialize geometry ###
geometrie = create_geometry_single_obstacle((37.5,31.5), 0.3)
#geometrie = Create_Geometry_Bottleneck(0.2, 0.8, 0.1, system_size)

### initialize crowd ###
menge = create_crowd(N, geometrie)

Init_Hom_Parameters!(p, menge)
Init_Hom_Vels!(menge, 1.0)

β, D, x_min, y_min, ϵ = π/2, sqrt(10), 0.0, 0.0, 0.001
Init_Two_Agents_α!(menge, β, D, x_min, y_min, ϵ)

D, x_min, y_min, ϵ = 3.0, 0.5, 1.5, 0.2
#Init_Three_Agents!(menge, D, x_min, y_min, ϵ)

v2, v1 = 0.1, 0.3
T2, T1 = 1.0, 1.0
D, ϵ = 3.0, 0.0
x_min, y_min = 2.5, 2.5

#Init_Overtaking!(menge, D, x_min, y_min, ϵ, v1, v2, T1, T2);

### Sparse Bottleneck ###

#Init_Agent_Bottleneck!(1.9*pi, 4, menge, system_size)

#ϕ1, ϕ2, dist1, dist2 = 2.5*π/2, 3.5*π/2, 4.0, 4.0
#Init_2_Agents_Bottleneck!(ϕ1, ϕ2, dist1, dist2, menge, system_size)

#ϕ1, ϕ2, ϕ3, dist1, dist2, dist3 = 0.0, 3*π/2, 3.2*π/2, 4.0, 4.0, 3.5
#Init_3_Agents_Bottleneck!(ϕ1, ϕ2, ϕ3, dist1, dist2, dist3, menge, system_size)


#ϕ1, ϕ2, ϕ3, ϕ4, dist1, dist2, dist3, dist4 = 1.2*π, π, 3.2*π/2, 0.0, 4.0, 4.0, 3.8, 4.0
#Init_4_Agents_Bottleneck!(ϕ1, ϕ2, ϕ3, ϕ4, dist1, dist2, dist3, dist4, menge, system_size)

In [63]:
positions, headings = Simulate!(menge, geometrie, sim_p[1], sim_p[2], sim_p[3], sim_p[4], sim_p[5], system_size);

In [64]:
using GR

GR.inline("mov")

j = 1

dt_save = sim_p[3]
geometry_x, geometry_y = rewrite_geometry(geometrie)
boundaries_x, boundaries_y = boundaries_rectangle_positions(system_size)

while j*sim_p[3] < sim_p[2]-sim_p[1]
    
        GR.clearws()

        GR.setwindow(-1, 6, -1, 6)
        GR.setviewport(0.0, 1, 0.0, 1)
    
        #plot_goal(goal1)
        #plot_goal(goal2)

        #plot_geometry(geometry_x, geometry_y, 3.6, 20)
        plot_boundaries(boundaries_x, boundaries_y, 0.2, 999)        
        
        plot_agents(positions[j, :], headings[j, :], 3.6, 100, 1.2, 999, 0.1)
                
        model = op_model
        timer = string("time ", round(j*dt_save, digits = 1), "s")
        GR.text(0.5, 0.95, timer)
        GR.text(0.5, 0.9, model)
        
        GR.updatews()
    
        j = j+1
end

In [65]:
GR.show()

GR.HTML("<video autoplay controls><source type=\"video/mp4\" src=\"data:video/mp4;base64,AAAAFGZ0eXBxdCAgAAACAHF0ICAAAAAId2lkZQAB9EFtZGF0AAACsQYF//+t3EXpvebZSLeWLNgg2SPu73gyNjQgLSBjb3JlIDE2MyAtIEguMjY0L01QRUctNCBBVkMgY29kZWMgLSBDb3B5bGVmdCAyMDAzLTIwMjEgLSBodHRwOi8vd3d3LnZpZGVvbGFuLm9yZy94MjY0Lmh0bWwgLSBvcHRpb25zOiBjYWJhYz0xIHJlZj0zIGRlYmxvY2s9MTowOjAgYW5hbHlzZT0weDM6MHgxMTMgbWU9aGV4IHN1Ym1lPTcgcHN5PTEgcHN5X3JkPTEuMDA6MC4wMCBtaXhlZF9yZWY9MSBtZV9yYW5nZT0xNiBjaHJvbWFfbWU9MSB0cmVsbGlzPTEgOHg4ZGN0PTEgY3FtPTAgZGVhZHpvbmU9MjEsMTEgZmFzdF9wc2tpcD0xIGNocm9tYV9xcF9vZmZzZXQ9LTIgdGhyZWFkcz0xMiBsb29rYWhlYWRfdGhyZWFkcz0yIHNsaWNlZF90aHJlYWRzPTAgbnI9MCBkZWNpbWF0ZT0xIGludGVybGFjZWQ9MCBibHVyYXlfY29tcGF0PTAgY29uc3RyYWluZWRfaW50cmE9MCBiZnJhbWVzPTMgYl9weXJhbWlkPTIgYl9hZGFwdD0xIGJfYmlhcz0wIGRpcmVjdD0xIHdlaWdodGI9MSBvcGVuX2dvcD0wIHdlaWdodHA9MiBrZXlpbnQ9MjUwIGtleWludF9taW49MjQgc2NlbmVjdXQ9NDAgaW50cmFfcmVmcmVzaD0wIHJjX2xvb2thaGVhZD00MCByYz1hYnIgbWJ0cmVlPTEgYml0cmF0ZT00MDAwIHJhdGV0b2w9MS4wIHFjb21wPTAuNjAgcXBtaW49MCBxcG1heD02OSBxcHN0ZXA9NCBpcF9yYXRpbz0xLjQwIGFxPTE6MS4wMACAAAAtL2WIhAAa//731LfMsu1Tb7YLqPeiniZfQ3UlAZuWxO06gAAAAwAAAwAAAwDpVIJjfO4A0Yy0DR+f2K4gDKdDocQKtjRy1EiAtdMSFpyuZYRVM6X0NlJ+oA8lHGo38VmUH95MU0d89tuB5s0Td5MylSissVBdG75/oP/3h/4+iG23qzuIA9SnVmc0e+25bbL5As4EO8db8eR4h2qSGs9U1eaXQKOQdHk/04vErZS5iwMOpop0GfsV+NlEAwDNeMst0csxcxQEiyOvirfAsRUoKvGId/kiC+3+1f27cWxh9qekVLJwDSk01BzOomvNBnCeVYx21viXJ7QDd9E2xJ0jRnnEuXAImMtKzedGYBW0DNW81d51UGbRvxruLmyf5szeYJBoFU7UZZ4CEiX6jBRS74LUSeNendbI50S4tU94PdBqv/vf4Q7d6vXjQmS5GxuwXA1J3HGfP/acP5BLOg+wSjTslSdVhQ092kDj2qRFAqnsWlah0HWkSNd+LMDTmPD7Vqf4zduEfC+7lIQCsrdknPL8n7S8364CCcEQopSuNguXiFSl1Rf7KE40QlbAFCFYFPCKDSrc+FiDLkhHDE4BFp7Kw+dfaXEkv36wh2dL+X99zO+k0Tg+Zrs/5Ys7t8oJdIKHhi2368PcFvYfWtfu9Ehsqn8xlqGJtUw9nhxFNOPfPJ0HhFJvsbf3o/LZ/fWHuK1r5+ln9s8S3JnLxIA/w+vI9kudhxcOUdiU93OEtoj7ra6FOfWFsH87+rRslaD75PDn4DgfQbLWtXz9m6KGAGdmdnQ4TopDhIfOBDjhOLW5EDmwjPJ33qRNzHtJcN4zj6zyuS7HeI8gjzniYPTCJBzRpj6sgWA3geC53yvallNnVmkH+m0slVaZQS2KJarOKT4G/MFkt/wYE0Jg6u+jwHodcch0bUgK5Ex937gdcGbV3zmkWeRjXxo98Y6rRrNgTthj1gwHzTYUPSpP/nFXhBWIJR0Nqd6vqskZM5E06H/X5/jLZNumB/VUAwjesbUox+LQJLb+T0MGk/amaCVRSFdhLl+konwySuYTxtkr1NGBX0dd/5dZwvnvy4y1z6DkS0B4oXnu9ZTCDwgA6AD3HfWt+zCdbL5wY+L2C75P1YTAHVqc/qIt+B0x9ftNPfufyHZrkD/8f9i6r+7nA1ht0qQx7/9q7x6SFMC1QVKQe3IPCSfjsfqUCFSm6yOCFWgp4bIoh5MFllvgGxxQlSagrxeSgQNysWu81TxieJ5WTpXWoW1N0j4LxS0bwFqiXryOt/UlZYwpsn7mh1zYBCLh9B+jgz0crbVKQHvgj6TIdaztyxpD1pS8sUveOCviW+QtoP6evvvbAv/r2uJ4rxGW1Jw8EF+NvlL9n4KOF4w+tASLuJ8JgxmLnV0p5fOwDyg5zy302NfBR7lxGyw9XJ07H+ErYNw8NsbyHEXfHLTwNAqlzgYQ11SQJoHRVz4DOIbZ6gSzQady+33YmH0oULO4XDA3aRlayoPejOfmbDQFdTRFY69u7q6hLnEHEEF0mkxDKeQVMCdwl4d6bQLmO26XGpcpCAwiCduvO1oUO345hKkgYX69lm4ClsOm6SLszbzt2GsU6GO9/i3ixK07RqwgKEm/CVRMUexSmYCOM0ZJqD3MhjkJ+KyxJoTrRy++GqCKYRKQ/gUCinUjix/KInvmEokaWAFDupotIVZm9S2w9w3NdzhxDroe3ArbH2V6sM94WB3SrC4sI2URuHW9V9SuoyOl9w1sQPxARoYo1bKIC3x9c6uzeKuY/iMFS06/xQurIDpQ2Wj9ptik3xO/wLgBPXMEK401wKf6SiighG+/5S+O1J8ggKVx/D7pQ9pYfw6Nc5pYrSErb2VUkIvuVmr/aYOhlJIjp3B0q8qTNTvnh+NZo40ZncwE9/1Ep5Zq+TN20QnZaltQ9YUfpnI+bvpiEsKqtKIReUzaVYLIFeFFqXzbexvZlKWZyX6pSOtaD+7V4hNvRcBtZpHkiKBEFF2PZWZ1kPsEupo4rIG6caQ0PYY7yFEluDWI6BaRrOGfFK1tlX8sTeGE0NmbFG4Zcre4ajYWjpnDJF5dhbZMFg1lx/AQr0GmFK6DB/1wsGToWV6kINPxrtOkRAYuQC9+88Z6/bwMa5BYikmwEPasw6ghlDJ9o+0YIa50CAc4YNVlAnvhJpOto7GPKAnTQ4Cbd3X+CV6FsRFvTOtezdEREXSXt5gDL1wUoW0bH5gN8mhBL4rd/z6A8s6pLT4DPmxxl/O0MmNA+alfyyKGKx1nlqaIQ54c15OrOVtqHpoLj5Vgg5EvCwSVKecI0hD6gYW35O332zv8OEZt/q1qGTOF2O/Ia7nRVxnSb4nM3Rb+eKcUVr3q+TzbAwN1XuqFdfDaZ8QN2s9qAc0WszJSY2sTJfPEzRwaVsldyqyT7CaRmTEkioiI0SFtl80I/9432iCSzoK7tCPWDMiyjIda/flY1mF53JsSceN/rJwQvxDySS1Ekqj99XhU4QIxek7J5ywve62raCmhnWt7OA2H+l77HjbspSKdtu3f0sznjLzCfSBP3tNpL8UJZj27jwyqbfp+2/rmspQLlHBbiTdBNmK1bShj1XxJTXAC80SEoIA7fDXjaLio8rhTi42y/dyYK22Nan5oLhzPW1Da4SlogyuBVY+tzagqRilj+TRxBCpkglHwAxngAhR8JbEk57X6tVh+vLllxp1mcgB7NQccNKOMrEYKvzLNbZPnApc82Kvmv8c3TMvXwkz2mJqKalMl2r8Splcl84aa+ht2xD/uxCBTRv6z7GRPeol/sSok+PsF29cdgCd8ohej4WZF0hJ7JgrzJ58dycP/Q3KexIUI2M/VMX3zncAtqllsW/lAMw52/dUREEgDF4xOHZlV5JJ/qUA4imd/RpYoqC5XXCgC1qHxCsZ2q0etjWmfj3IXxuJ

In [ ]:
Path = "/home/jakob/Dokumente/Sim_Data/"
dir = "90_CollisionFreeSpeed"
header = Header(p, p_desc, sim_p, sim_p_desc, op_model, tact_model, order, update, N, system_size);

Save_Data!(Path, dir, header, positions)

In [ ]:
samples = 15

for i in 1:samples
    
    Random.seed!()
    Init_Hom_Vels!(menge, 0.0)
    
    β, D, x_min, y_min, ϵ = pi, i/3, 2.5, 5.0, 0.00
    Init_Two_Agents_α!(menge, β, D, x_min, y_min, ϵ)

    
    positions, headings, ttcs = Simulate!(menge, geometrie, sim_p[1], sim_p[2], sim_p[3], sim_p[4], sim_p[5], system_size);
    Save_Data!(Path, dir, header, positions)
    
end

In [ ]:
name = "Agents_Test"
path = "/home/jakob/Dokumente/Renne_Software/UMANS/examples/agents/"

Write_Agents_XML(menge, name, path)